# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**    

Таблица `books`    
Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`    
Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`    
Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`    
Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`    
Содержит данные о пользовательских обзорах:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

**Схема данных**

![изображение](https://drive.google.com/uc?export=view&id=1fYKcfRgYHQb_oBch0y9gpUa-7malT6Ub)

https://drive.google.com/file/d/1fYKcfRgYHQb_oBch0y9gpUa-7malT6Ub/view?usp=share_link


**Задания**

1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Как выполнить задание?**

- Опишите цели исследования;
- Исследуйте таблицы — выведите первые строки;
- Сделайте по одному SQL-запросу для решения каждого задания;
- Выведите результаты каждого запроса в тетрадке;
- Опишите выводы по каждой из решённых задач.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Цель

- Ознакомиться с базой данных действующего бизнеса.
- Провести предобработку данных.
- Выявить популярные издательства, авторов и книги.
- Опредилить направление развития для нового продукта

## 1 Предобработка данных
### 1.1 Таблица с данными о книгах

In [3]:
# обратимся к таблице books
# запишем все поля в одноимённый датафрейм
query = ''' SELECT * FROM books ''' 
books = pd.io.sql.read_sql(query, con = engine)

In [4]:
display(books.info())
display(books.head(2))
print('Кол-во дубликатов =', books.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


Кол-во дубликатов = 0


Пропусков, дубликатов нет.    

### 1.2 Таблица с данными об авторах

In [5]:
# обратимся к таблице authors
# запишем все поля в одноимённый датафрейм
query = ''' SELECT * FROM authors ''' 
authors = pd.io.sql.read_sql(query, con = engine)

In [6]:
display(authors.info())
display(authors.head(2))
print('Кол-во дубликатов =', authors.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


Кол-во дубликатов = 0


Пропусков, дубликатов нет.    
### 1.3 Таблица с данными об издательствах

In [7]:
# обратимся к таблице publishers
# запишем все поля в одноимённый датафрейм
query = ''' SELECT * FROM publishers ''' 
publishers = pd.io.sql.read_sql(query, con = engine)

In [8]:
display(publishers.info())
display(publishers.head(2))
print('Кол-во дубликатов =', publishers.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

,publisher_id,publisher
0,1,Ace
1,2,Ace Book


Кол-во дубликатов = 0


### 1.4 Таблица с данными о пользовательских оценках книг

In [9]:
# обратимся к таблице ratings
# запишем все поля в одноимённый датафрейм
query = ''' SELECT * FROM ratings ''' 
ratings = pd.io.sql.read_sql(query, con = engine)

In [10]:
display(ratings.info())
display(ratings.head(2))
print('Кол-во дубликатов =', ratings.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


Кол-во дубликатов = 0


### 1.5 Таблица с данными о пользовательских обзорах

In [11]:
# обратимся к таблице reviews
# запишем все поля в одноимённый датафрейм
query = ''' SELECT * FROM reviews ''' 
reviews = pd.io.sql.read_sql(query, con = engine)

In [12]:
display(ratings.info())
display(ratings.head(2))
print('Кол-во дубликатов =', reviews.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


Кол-во дубликатов = 0


## 2 Задание
### 2.1 Посчитайте, сколько книг вышло после 1 января 2000 года

In [13]:
query = ''' 
    SELECT COUNT(book_id) 
    FROM books
    WHERE publication_date > '2000-01-01'
    ''' 
data_1 = pd.io.sql.read_sql(query, con = engine)

In [14]:
print('Кол-во книг, вышедших после 1 января 2000 года =', data_1['count'].loc[data_1.index[0]])

Кол-во книг, вышедших после 1 января 2000 года = 819


Из 1000 книг, 819 вышло после 1 января 2000 года

### 2.2 Для каждой книги посчитайте количество обзоров и среднюю оценку

In [15]:
query = ''' 
    SELECT 
        b.book_id, 
        b.title, 
        COUNT(re.review_id) AS reviews,
        ra.avg_rating
    FROM books AS b
    LEFT JOIN 
            (SELECT
                book_id,
                AVG(rating) AS avg_rating
             FROM ratings
             GROUP BY book_id) AS ra ON ra.book_id = b.book_id
    LEFT JOIN reviews AS re ON b.book_id = re.book_id
    
    GROUP BY b.book_id, ra.avg_rating
    ORDER BY reviews DESC
    ''' 
data_2 = pd.io.sql.read_sql(query, con = engine)

In [16]:
print('Количество обзоров и среднея оценка')
display(data_2[['title','reviews', 'avg_rating']])

Количество обзоров и среднея оценка


,title,reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Alchemist,6,3.789474
2,Outlander (Outlander #1),6,4.125000
3,The Catcher in the Rye,6,3.825581
4,The Giver (The Giver #1),6,3.750000
...,...,...,...
995,The Natural Way to Draw,0,3.000000
996,Leonardo's Notebooks,0,4.000000
997,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


In [17]:
# выполним проверку
print('Кол-во книг =', len(books))
print('Кол-во обзоров (Python ver.) =', len(reviews))
print('Кол-во обзоров (SQL ver.) =', data_2.reviews.sum())

Кол-во книг = 1000
Кол-во обзоров (Python ver.) = 2793
Кол-во обзоров (SQL ver.) = 2793


Собрали сводную таблицу для анализа

### 2.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [18]:
query = ''' 
    SELECT 
        p.publisher_id, 
        p.publisher, 
        COUNT(b.book_id) AS number_of_books
    FROM publishers AS p
    JOIN books AS b ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher_id
    ORDER BY number_of_books DESC
    LIMIT 2
    ''' 
data_3 = pd.io.sql.read_sql(query, con = engine)

In [19]:
print('Издательство, выпустившее наибольшее число книг -', 
      data_3['publisher'].loc[data_3.index[0]],
      data_3['number_of_books'].loc[data_3.index[0]], 'шт.')

Издательство, выпустившее наибольшее число книг - Penguin Books 42 шт.


### 2.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [20]:
query = ''' 
WITH tmp AS (
    SELECT
        a.author,
        r.book_id,
        --COUNT(book_id) AS number_of_books,
        AVG(r.rating) AS avg_rating
    FROM ratings AS r
    JOIN books AS b ON b.book_id = r.book_id
    JOIN authors AS a ON a.author_id = b.author_id
    GROUP BY r.book_id, a.author
    HAVING COUNT(r.book_id) >= 50
    ORDER BY avg_rating DESC
    ) 
    SELECT 
        author,
        AVG(avg_rating) AS avg_rating
    FROM tmp
    GROUP BY author
    ''' 
data_4 = pd.io.sql.read_sql(query, con = engine)

In [21]:
print('Автор с самой высокой средней оценкой книги')
display(data_4.head(1))

Автор с самой высокой средней оценкой книги


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


### 2.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [22]:
query = ''' 
WITH user_ratings AS (
  SELECT
    username,
    COUNT(username) AS number_of_ratings
  FROM
    ratings
  GROUP BY
    username
  HAVING
    COUNT(username) > 50
),
user_reviews AS (
  SELECT
    username,
    COUNT(username) AS number_of_reviews
  FROM
    reviews
  GROUP BY
    username
)
SELECT
  AVG(number_of_reviews)
FROM
  user_ratings
  INNER JOIN user_reviews ON user_ratings.username = user_reviews.username
    ''' 
data_5 = pd.io.sql.read_sql(query, con = engine)

In [23]:
print('Среднее количество обзоров от пользователей, которые поставили больше 50 оценок =', 
      data_5['avg'].loc[data_5.index[0]])

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок = 24.333333333333332


## 3 Вывод

- В полученной баде 1000 книг. Из них 819 вышло после 1 января 2000 года
- Пользователи оставили 2793 обзоров. Книга Twilight получила больше всех обзоров (7 ревью)
- Penguin Books выпустило больше всех книг (42 шт.)
- Самую высокую среднею оценку (4.3) получил автор J.K. Rowling/Mary GrandPré
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок = 24.3